In [1]:
import timm
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from resnet_generator import Generator, Critic
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
from transformer import Attention
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
from crd.criterion import CRDLoss
from dataset.imagenet import get_dataloader_sample
from timm.data import transforms_factory

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
model_ns = model_ns.cuda()
model_ns = torch.nn.DataParallel(model_ns)
model_ns = model_ns.eval()

In [9]:
# fc_name = 'module.classifier.'
# teacher_model_weights = {}
# for name, param in model_ns.named_parameters():
#     teacher_model_weights[name] = param.detach()

In [10]:
model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
model_raw = model_raw.cuda()
model_raw = torch.nn.DataParallel(model_raw)

In [11]:
model_g = Generator(args, img_size=args.img_size, max_conv_dim=256)
model_g = model_g.cuda()

In [12]:
model_critic = Critic(args)
model_critic = model_critic.cuda()
model_critic = torch.nn.DataParallel(model_critic)

In [13]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
    model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [14]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

In [15]:
model_g = torch.nn.DataParallel(model_g)

In [16]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [17]:
num_aug_splits = 0

In [18]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP

In [19]:
# train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [20]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 3662851


In [21]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [22]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [23]:
transform = transforms_factory.transforms_imagenet_train(img_size=args.img_size, auto_augment=False, color_jitter=args.color_jitter, interpolation=args.interpolation, re_count=args.recount, re_mode=args.remode, re_prob=args.reprob)

In [24]:
train_loader = get_dataloader_sample(transform, dataset='imagenet', batch_size=args.batch_size, is_sample=True, k=4096)

stage1 finished!
dataset initialized!
num_samples 1281167
num_class 1000


In [25]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [26]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn
    
crd_loss_fn = CRDLoss(args).cuda()
critic_loss_fn = nn.BCELoss()
real_label = 1
fake_label = 0

In [27]:
trainable_list = nn.ModuleList([])
trainable_list.append(model_g)
trainable_list.append(model_raw)
trainable_list.append(crd_loss_fn.embed_s)
trainable_list.append(crd_loss_fn.embed_t)

ModuleList(
  (0): DataParallel(
    (module): Generator(
      (from_rgb): Conv2d(3, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (encode): ModuleList(
        (0): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(68, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(68, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (norm2): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (conv1x1): Conv2d(68, 136, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(136, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(136, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(136, eps=1e-05

In [28]:
optimizer = create_optimizer(args, trainable_list)
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [29]:
optimizer_critic = create_optimizer(args, model_critic)
lr_scheduler_critic, _ = create_scheduler(args, optimizer_critic)

In [30]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [31]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [32]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [33]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [34]:
def train_epoch(epoch, model_g, model_raw, model_ns, model_critic, loader, optimizer, optimizer_critic, loss_fn, crd_loss_fn, critic_loss_fn, args,
               lr_scheduler=None, lr_scheduler_critic=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_crd = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    losses_critic = AverageMeter()
    losses_g = AverageMeter()
    
    model_g.train()
    model_critic.train()
    model_raw.train()
    model_ns.eval()
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
        
    for batch_idx, data in enumerate(loader):
        inputs, target, index, contrast_idx = data
        inputs, target, index, contrast_idx = inputs.cuda(), target.cuda(), index.cuda(), contrast_idx.cuda()
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        
        g_out = model_g(inputs)
        
        inputs_z = inputs + g_out
        output, f_raw = model_raw(inputs_z)
        with torch.no_grad():
#             inputs_ns = F.interpolate(inputs, size=300, mode='bicubic')
            out_ns, f_ns = model_ns(inputs)
            out_ns = out_ns.detach()
        
        # Critic
        model_critic.train()
        model_critic.zero_grad()
        label = torch.full((f_raw.size(0), ), real_label, device='cuda')
        critic_out = model_critic(f_ns).view(-1)
        loss_critic_ns = critic_loss_fn(critic_out, label)
        loss_critic_ns.backward()
        loss_critic_ns_out = loss_critic_ns.item()
        
        label.fill_(fake_label)
        critic_out = model_critic(f_raw.detach()).view(-1)
        loss_critic_raw = critic_loss_fn(critic_out, label)
        loss_critic_raw.backward()
        loss_critic_raw_out = loss_critic_raw.item()
        loss_critic = loss_critic_ns_out + loss_critic_raw_out
        optimizer_critic.step()
        
        # KD
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
       # CE
        loss_ce = loss_fn(output, target)
        
        # CRD
        loss_crd = crd_loss_fn(f_raw, f_ns, index, contrast_idx)
        
        # G
        model_critic.eval()
        label.fill_(real_label)
        critic_g_out = model_critic(f_raw).view(-1)
        loss_g = critic_loss_fn(critic_g_out, label)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce + args.lambda_crd * loss_crd + loss_g
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_crd.update(loss_crd.item(), inputs.size(0))
            losses_critic.update(loss_critic, inputs.size(0))
            losses_g.update(loss_g.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Loss_crd: {loss_crd.val:>9.6f} ({loss_crd.avg:>6.4f})  '
                    'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_critic: {loss_critic.val:>9.6f} ({loss_critic.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.7f}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        loss_crd=losses_crd,
                        loss_g=losses_g,
                        loss_critic=losses_critic,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model_raw, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)
            
        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)
            lr_scheduler_critic.step_update(num_updates=num_updates, metric=losses_critic.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()
        

    return OrderedDict([('loss', losses_m.avg)])

In [35]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
#     model_att.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            
            out = model(inputs)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, _ = model_raw(inputs_out)
#             output = model_att(inputs_out, output)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [ ]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_ns, model_critic, train_loader, optimizer, optimizer_critic, 
        train_loss_fn, crd_loss_fn, critic_loss_fn, args,
        lr_scheduler=lr_scheduler, lr_scheduler_critic=lr_scheduler_critic, 
        saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary( 
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)
        torch.save(model_raw, 'raw')
        torch.save(model_critic, 'critic')

/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


normalization constant Z_v1 is set to 2831728.0
normalization constant Z_v2 is set to 2842864.0


INFO:root:Train: 1 [   0/8374 (  0%)]  Loss: 21.028940 (21.0289)  Loss_kd:  0.532949 (0.5329)  Loss_ce:  3.193007 (3.1930)  Loss_crd: 20.722170 (20.7222)  Loss_g:  0.725248 (0.7252)  Loss_critic:  1.417938 (1.4179)  Time: 24.072s,   19.07/s  (24.072s,   19.07/s)  LR: 0.0100000  Data: 5.422 (5.422)
INFO:root:Train: 1 [  50/8374 (  1%)]  Loss: 19.675608 (19.7001)  Loss_kd:  0.389105 (0.3828)  Loss_ce:  1.981173 (2.1080)  Loss_crd: 20.578053 (20.4913)  Loss_g:  0.842888 (0.8163)  Loss_critic:  1.150489 (1.2411)  Time: 1.176s,  390.41/s  (1.659s,  276.60/s)  LR: 0.0100000  Data: 0.013 (0.119)
INFO:root:Train: 1 [ 100/8374 (  1%)]  Loss: 19.489040 (19.6058)  Loss_kd:  0.397458 (0.3817)  Loss_ce:  2.005741 (2.0818)  Loss_crd: 20.416481 (20.4803)  Loss_g:  0.752655 (0.7580)  Loss_critic:  1.813709 (1.4150)  Time: 1.188s,  386.21/s  (1.428s,  321.45/s)  LR: 0.0100000  Data: 0.013 (0.067)
INFO:root:Train: 1 [ 150/8374 (  2%)]  Loss: 19.994667 (19.6123)  Loss_kd:  0.431213 (0.3875)  Loss_ce:  2.

INFO:root:Train: 1 [1400/8374 ( 17%)]  Loss: 18.818636 (19.2411)  Loss_kd:  0.415783 (0.4375)  Loss_ce:  2.375732 (2.2874)  Loss_crd: 19.984623 (20.3563)  Loss_g:  0.039423 (0.2313)  Loss_critic: 13.604064 (11.5019)  Time: 1.182s,  388.35/s  (1.199s,  382.70/s)  LR: 0.0100000  Data: 0.013 (0.017)
INFO:root:Train: 1 [1450/8374 ( 17%)]  Loss: 18.750465 (19.2324)  Loss_kd:  0.450237 (0.4367)  Loss_ce:  2.303662 (2.2870)  Loss_crd: 19.930775 (20.3503)  Loss_g:  0.051946 (0.2285)  Loss_critic: 16.697525 (11.5447)  Time: 1.171s,  391.94/s  (1.199s,  382.95/s)  LR: 0.0100000  Data: 0.013 (0.017)
INFO:root:Train: 1 [1500/8374 ( 18%)]  Loss: 18.951246 (19.2230)  Loss_kd:  0.416260 (0.4370)  Loss_ce:  2.378465 (2.2863)  Loss_crd: 20.100838 (20.3441)  Loss_g:  0.075849 (0.2245)  Loss_critic: 12.284160 (11.6823)  Time: 1.173s,  391.32/s  (1.198s,  383.14/s)  LR: 0.0100000  Data: 0.013 (0.017)
INFO:root:Train: 1 [1550/8374 ( 19%)]  Loss: 19.092564 (19.2131)  Loss_kd:  0.438700 (0.4368)  Loss_ce:  2

INFO:root:Train: 1 [2800/8374 ( 33%)]  Loss: 18.951273 (19.0556)  Loss_kd:  0.434765 (0.4414)  Loss_ce:  2.360312 (2.2934)  Loss_crd: 19.926834 (20.1890)  Loss_g:  0.214730 (0.1695)  Loss_critic: 11.995063 (14.0446)  Time: 1.177s,  389.96/s  (1.190s,  385.57/s)  LR: 0.0100000  Data: 0.013 (0.015)
INFO:root:Train: 1 [2850/8374 ( 34%)]  Loss: 18.678247 (19.0508)  Loss_kd:  0.496733 (0.4416)  Loss_ce:  2.307420 (2.2945)  Loss_crd: 19.751501 (20.1826)  Loss_g:  0.072893 (0.1685)  Loss_critic: 21.805289 (14.0728)  Time: 1.175s,  390.56/s  (1.190s,  385.65/s)  LR: 0.0100000  Data: 0.013 (0.015)
INFO:root:Train: 1 [2900/8374 ( 35%)]  Loss: 19.197048 (19.0488)  Loss_kd:  0.522961 (0.4424)  Loss_ce:  2.681200 (2.2969)  Loss_crd: 19.852974 (20.1762)  Loss_g:  0.110508 (0.1685)  Loss_critic: 17.192480 (14.0947)  Time: 1.175s,  390.74/s  (1.190s,  385.70/s)  LR: 0.0100000  Data: 0.013 (0.015)
INFO:root:Train: 1 [2950/8374 ( 35%)]  Loss: 18.890476 (19.0456)  Loss_kd:  0.425237 (0.4428)  Loss_ce:  2

...

INFO:root:Train: 1 [8300/8374 ( 99%)]  Loss: 18.529005 (18.8341)  Loss_kd:  0.608237 (0.4814)  Loss_ce:  3.175880 (2.5996)  Loss_crd: 18.377851 (19.5265)  Loss_g:  0.042607 (0.1319)  Loss_critic: 17.075572 (17.5501)  Time: 1.175s,  390.71/s  (1.184s,  387.63/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 1 [8350/8374 (100%)]  Loss: 18.949936 (18.8352)  Loss_kd:  0.596438 (0.4820)  Loss_ce:  3.472723 (2.6051)  Loss_crd: 18.516731 (19.5207)  Loss_g:  0.067390 (0.1316)  Loss_critic: 14.373836 (17.5365)  Time: 1.186s,  387.17/s  (1.184s,  387.63/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 1 [8373/8374 (100%)]  Loss: 18.878410 (18.8352)  Loss_kd:  0.597585 (0.4823)  Loss_ce:  3.363386 (2.6069)  Loss_crd: 18.488459 (19.5180)  Loss_g:  0.126672 (0.1316)  Loss_critic: 13.444145 (17.5244)  Time: 7.205s,   40.81/s  (1.185s,  248.14/s)  LR: 0.0100000  Data: 0.008 (0.014)
INFO:root:Test: [   0/327]  Time: 7.410s (7.410s,   20.65/s)  Loss:  1.5958 (1.5958)  Acc@1:  71.242 ( 71.

INFO:root:Train: 2 [ 450/8374 (  5%)]  Loss: 17.286768 (19.5994)  Loss_kd:  0.503846 (0.6045)  Loss_ce:  3.207016 (3.4027)  Loss_crd: 16.875326 (19.3086)  Loss_g:  0.075646 (0.1453)  Loss_critic: 29.743185 (27.0368)  Time: 1.189s,  386.11/s  (1.211s,  378.94/s)  LR: 0.0100000  Data: 0.013 (0.023)
INFO:root:Train: 2 [ 500/8374 (  6%)]  Loss: 16.855762 (19.3621)  Loss_kd:  0.555714 (0.6018)  Loss_ce:  2.957006 (3.3751)  Loss_crd: 16.609770 (19.0559)  Loss_g:  0.055228 (0.1404)  Loss_critic: 27.206404 (27.2200)  Time: 1.186s,  386.91/s  (1.209s,  379.51/s)  LR: 0.0100000  Data: 0.013 (0.022)
INFO:root:Train: 2 [ 550/8374 (  7%)]  Loss: 16.924164 (19.1353)  Loss_kd:  0.572648 (0.5984)  Loss_ce:  3.142022 (3.3423)  Loss_crd: 16.431767 (18.8242)  Loss_g:  0.064080 (0.1353)  Loss_critic: 22.614855 (27.1155)  Time: 1.200s,  382.53/s  (1.208s,  379.95/s)  LR: 0.0100000  Data: 0.013 (0.022)
INFO:root:Train: 2 [ 600/8374 (  7%)]  Loss: 16.991049 (18.9335)  Loss_kd:  0.561135 (0.5959)  Loss_ce:  2

INFO:root:Train: 2 [1800/8374 ( 21%)]  Loss: 21.259764 (18.3218)  Loss_kd:  0.658104 (0.5968)  Loss_ce:  6.900223 (3.9349)  Loss_crd: 17.126774 (17.0710)  Loss_g:  0.000017 (0.1333)  Loss_critic: 46.472806 (38.0886)  Time: 1.186s,  386.95/s  (1.200s,  382.55/s)  LR: 0.0100000  Data: 0.013 (0.016)
INFO:root:Train: 2 [1850/8374 ( 22%)]  Loss: 21.015694 (18.4001)  Loss_kd:  0.641757 (0.5986)  Loss_ce:  6.890121 (4.0149)  Loss_crd: 16.851742 (17.0702)  Loss_g:  0.002423 (0.1305)  Loss_critic: 28.787366 (37.8708)  Time: 1.194s,  384.36/s  (1.200s,  382.61/s)  LR: 0.0100000  Data: 0.013 (0.016)
INFO:root:Train: 2 [1900/8374 ( 23%)]  Loss: 21.726221 (18.4734)  Loss_kd:  0.673366 (0.6000)  Loss_ce:  6.893696 (4.0906)  Loss_crd: 17.586487 (17.0691)  Loss_g:  0.089969 (0.1276)  Loss_critic: 17.927829 (37.5389)  Time: 1.197s,  383.45/s  (1.200s,  382.63/s)  LR: 0.0100000  Data: 0.013 (0.015)
INFO:root:Train: 2 [1950/8374 ( 23%)]  Loss: 21.089409 (18.5405)  Loss_kd:  0.681916 (0.6014)  Loss_ce:  6

INFO:root:Train: 2 [3200/8374 ( 38%)]  Loss: 20.366098 (19.3787)  Loss_kd:  0.693317 (0.6284)  Loss_ce:  6.559687 (5.1513)  Loss_crd: 16.391369 (16.8969)  Loss_g:  0.000000 (0.0815)  Loss_critic: 100.397798 (46.6214)  Time: 1.199s,  382.67/s  (1.199s,  382.97/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 2 [3250/8374 ( 39%)]  Loss: 20.286169 (19.3919)  Loss_kd:  0.663503 (0.6292)  Loss_ce:  6.527094 (5.1722)  Loss_crd: 16.369465 (16.8878)  Loss_g:  0.000000 (0.0802)  Loss_critic: 100.429291 (47.4491)  Time: 1.192s,  384.91/s  (1.199s,  382.98/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 2 [3300/8374 ( 39%)]  Loss: 20.144520 (19.4043)  Loss_kd:  0.679138 (0.6300)  Loss_ce:  6.485237 (5.1924)  Loss_crd: 16.225180 (16.8785)  Loss_g:  0.000000 (0.0790)  Loss_critic: 100.287060 (48.2506)  Time: 1.190s,  385.84/s  (1.198s,  382.98/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 2 [3350/8374 ( 40%)]  Loss: 19.998583 (19.4157)  Loss_kd:  0.663861 (0.6309)  Loss_ce:

INFO:root:Train: 2 [4600/8374 ( 55%)]  Loss: 19.905861 (19.5321)  Loss_kd:  0.775665 (0.6495)  Loss_ce:  6.330792 (5.5161)  Loss_crd: 15.942432 (16.6333)  Loss_g:  0.045457 (0.0598)  Loss_critic: 16.565355 (54.5311)  Time: 1.188s,  386.25/s  (1.198s,  383.16/s)  LR: 0.0100000  Data: 0.013 (0.014)
INFO:root:Train: 2 [4650/8374 ( 56%)]  Loss: 19.874672 (19.5335)  Loss_kd:  0.687891 (0.6504)  Loss_ce:  6.417292 (5.5241)  Loss_crd: 15.923868 (16.6242)  Loss_g:  0.030395 (0.0596)  Loss_critic: 24.177089 (54.1769)  Time: 1.190s,  385.87/s  (1.198s,  383.17/s)  LR: 0.0100000  Data: 0.013 (0.014)
/home/cutz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
INFO:root:Train: 2 [4700/8374 ( 56%)]  Loss: 19.566723 (19.5353)  Loss_kd:  0.699164 (0.6505)  Loss_ce:  6.380479 (5.5332)  Loss_crd: 15.566801 (16.6151)  Loss_g:  0.033640 (0.0594)  Loss_critic: 17.957432 (53.8285)  Time: 1.